# Taller: Modelación del Transporte de Contaminantes en Canales Abiertos
## Notebook 0: Introducción y configuración del entorno

<img src="../resources/img/header_contaminantes.png" width="800" alt="Transporte de contaminantes en canal abierto">

## Objetivos del taller

En este taller práctico, implementaremos un modelo numérico para simular el transporte de contaminantes en canales abiertos mediante la ecuación de advección-difusión-reacción.

**Al finalizar el taller, serás capaz de:**
- Implementar un esquema numérico de diferencias finitas para la ecuación de transporte
- Simular diferentes escenarios de contaminación en canales
- Analizar la estabilidad y precisión de la solución numérica
- Visualizar y evaluar resultados de simulaciones

## Estructura del taller

Este taller está organizado en una serie de notebooks interactivos que te guiarán en la implementación paso a paso:

0. **Introducción** (este notebook)
   - Configuración del entorno
   - Problema a modelar
   - Conceptos básicos

1. **Fundamentos y visualización de términos**
   - Ecuación de advección-difusión-reacción
   - Efectos de cada término
   - Simulación de casos simples

2. **Adimensionalización**
   - Importancia de trabajar con variables adimensionales
   - Identificación de parámetros adimensionales clave
   - Implementación de la forma adimensional

3. **Implementación del esquema numérico**
   - Discretización por diferencias finitas
   - Esquema explícito FTCS
   - Condiciones iniciales y de contorno

4. **Análisis de estabilidad**
   - Criterios de estabilidad numérica
   - Experimentación con parámetros
   - Efecto en la solución

5. **Caso práctico: vertido en canal de riego**
   - Simulación de escenario realista
   - Análisis e interpretación de resultados
   - Visualizaciones avanzadas

## Verificación del entorno de trabajo

Para este taller, utilizaremos varias bibliotecas científicas de Python. Ejecuta la siguiente celda para verificar que todo esté correctamente instalado:

## El problema del transporte de contaminantes

Imaginemos un escenario práctico: un vertido accidental de un contaminante en un canal de riego. ¿Cómo se dispersará este contaminante a lo largo del canal? ¿Cuándo llegará a ciertos puntos críticos aguas abajo? ¿Qué concentraciones máximas podemos esperar?

Para responder a estas preguntas, necesitamos un modelo matemático que describa el transporte del contaminante. La ecuación que gobierna este fenómeno es la **ecuación de advección-difusión-reacción**:

$$\frac{\partial C}{\partial t} + U\frac{\partial C}{\partial x} = D\frac{\partial^2 C}{\partial x^2} - kC$$

Donde:
- $C(x,t)$ es la concentración del contaminante [mg/L]
- $t$ es el tiempo [s]
- $x$ es la posición a lo largo del canal [m]
- $U$ es la velocidad media del flujo [m/s]
- $D$ es el coeficiente de dispersión longitudinal [m²/s]
- $k$ es la constante de degradación [1/s]

### Interpretación física de los términos

Esta ecuación captura tres procesos físicos fundamentales:

1. **Advección** ($U\frac{\partial C}{\partial x}$): Transporte debido al movimiento del agua
2. **Difusión/Dispersión** ($D\frac{\partial^2 C}{\partial x^2}$): Esparcimiento debido a gradientes de concentración
3. **Reacción** ($kC$): Degradación del contaminante por procesos químicos o biológicos

## Demostración interactiva: Efectos básicos de los parámetros

Antes de adentrarnos en la implementación numérica, exploremos visualmente cómo los parámetros $U$, $D$ y $k$ afectan el comportamiento de un pulso de contaminante:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

def gaussian_pulse(x, mu, sigma):
    """Función gaussiana para representar un pulso de contaminante"""
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

@widgets.interact(
    velocity=widgets.FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Velocidad (m/s):'),
    dispersion=widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description='Dispersión (m²/s):'),
    reaction=widgets.FloatSlider(min=0.0, max=0.05, step=0.001, value=0.0, description='Degradación (1/s):'),
    time=widgets.FloatSlider(min=0, max=100, step=5, value=50, description='Tiempo (s):')
)
def simple_transport_demo(velocity, dispersion, reaction, time):
    """Demo simplificada del transporte de un pulso de contaminante"""
    # Parámetros
    L = 100  # longitud del canal [m]
    nx = 200  # puntos de la malla
    x = np.linspace(0, L, nx)
    
    # Condición inicial: pulso gaussiano en x=20
    sigma_0 = 5  # desviación estándar inicial [m]
    mu_0 = 20    # posición inicial [m]
    
    # Solución analítica aproximada para un pulso gaussiano
    mu_t = mu_0 + velocity * time  # Posición del centro (advección)
    sigma_t = np.sqrt(sigma_0**2 + 2 * dispersion * time)  # Ensanchamiento (dispersión)
    amplitude = np.exp(-reaction * time)  # Decaimiento (reacción)
    
    # Calcular la solución actual y la inicial
    C_inicial = gaussian_pulse(x, mu_0, sigma_0)
    C_actual = amplitude * gaussian_pulse(x, mu_t, sigma_t)
    
    # Visualización
    plt.figure(figsize=(12, 6))
    plt.plot(x, C_inicial, 'k--', alpha=0.5, label='Condición inicial')
    plt.plot(x, C_actual, 'b-', linewidth=2, label=f'Después de {time} segundos')
    plt.xlabel('Distancia a lo largo del canal (m)')
    plt.ylabel('Concentración relativa')
    plt.title(f'Transporte de contaminante con U={velocity} m/s, D={dispersion} m²/s, k={reaction} 1/s')
    plt.xlim(0, L)
    plt.ylim(0, 1.1)
    plt.legend()
    plt.grid(True)
    
    # Mostrar información sobre el comportamiento
    print(f"Desplazamiento del pico: {velocity*time:.1f} m")
    print(f"Ensanchamiento de la curva: de σ={sigma_0:.1f} m a σ={sigma_t:.1f} m")
    if reaction > 0:
        print(f"Reducción de la concentración máxima: {(1-amplitude)*100:.1f}%")
    
    plt.show()

## Plan de trabajo para el taller

En este taller, desarrollaremos paso a paso un modelo numérico completo para simular el transporte de contaminantes. A continuación, se presenta un resumen de las actividades que realizaremos:

1. **Explorar visualmente** los efectos de cada término de la ecuación
2. **Adimensionalizar** la ecuación para trabajar con parámetros adimensionales clave
3. **Implementar** el método de diferencias finitas para resolver numéricamente la ecuación
4. **Analizar** la estabilidad numérica y determinar los parámetros óptimos
5. **Simular** un caso práctico con parámetros realistas
6. **Visualizar e interpretar** los resultados desde una perspectiva ingenieril

### Recomendaciones para el trabajo

- Ejecuta las celdas en orden secuencial
- Tómate tiempo para comprender cada paso antes de continuar
- Experimenta modificando parámetros para ver qué ocurre
- Completa los ejercicios marcados con el texto "EJERCICIO:"
- Guarda periódicamente tu trabajo

## Preparación para el siguiente notebook

En el próximo notebook, comenzaremos explorando en detalle cada término de la ecuación de advección-difusión-reacción y visualizando sus efectos por separado. Para prepararte, piensa en las siguientes preguntas:

1. ¿Qué efecto tendría la advección por sí sola sobre un pulso de contaminante?
2. ¿Cómo cambiaría la forma del pulso si solo actuara la dispersión?
3. ¿Qué sucede con la masa total del contaminante cuando hay reacción?

<div style="background-color: #e6f3ff; padding: 15px; border-radius: 5px; border-left: 5px solid #4285f4;">
    <h3 style="margin-top: 0; color: #4285f4;">¡Manos a la obra!</h3>
    <p>Ahora que tienes una visión general del problema y has verificado tu entorno de trabajo, puedes continuar con el <b>Notebook 1: Fundamentos y visualización de términos</b>.</p>
</div>